In [91]:
import pandas as pd
import numpy as np
import sys
import re

In [92]:
# pip install soynlp

In [93]:
# pdf 추출 모듈 import

%run module_pdf_extractor.ipynb

In [94]:
# 이미 추출되어 있는 공시데이터 텍스트 가져오기

path = './1-2. ESG report/'
file_list = os.listdir(path+'output/')
file_list = [file for file in file_list if file.startswith("ESG_text")]

In [95]:
# 각 공시 보고서별 e/s/g 페이지 번호 미리 저장
# e/s/g 순서

dict = {
    'GSretail_2023':[range(7,18), range(18,34),range(2,7)],
    'hyundaehomeshopping_2023' : [range(6,16), range(16,32), range(2,6)],
    'LGlife_2023' : [range(5,22), range(22,53), range(2,5)],
    'lottehi_2023' : [range(5,14), range(14,32), range(2,5)],
    'sinsegye_2023' : [range(6,18), range(18,35), range(2,6)],
}

e/s/g별 텍스트 모두 모아서 각각 텍스트 파일로 저장

In [97]:
e = ''
s = ''
g = ''

In [98]:
paths = []

for file in file_list:
    name = file.replace('ESG_text_','').replace('.csv','')
    data = json_to_csv(path, '', name)
    globals()[f'{name}_e'] = ''
    globals()[f'{name}_s'] = ''
    globals()[f'{name}_g'] = ''


    for j in range(len(data["elements"])):
        if 'Text' in data["elements"][j]:
            page = data["elements"][j]['Page']
            if page in dict[name][0]:
                    globals()[f'{name}_e'] = globals()[f'{name}_e'] + data["elements"][j]['Text'] + ' '
            elif page in dict[name][1]:
                    globals()[f'{name}_s'] = globals()[f'{name}_s'] + data["elements"][j]['Text'] + ' '
            elif page in dict[name][2]:
                    globals()[f'{name}_g'] = globals()[f'{name}_g'] + data["elements"][j]['Text'] + ' '
            else:
                pass
        else:
            pass

    p = path +'output/'+f'{name}_e_doublespacelinecorpus.txt'
    paths.append(p)
    file = open(p, 'w', encoding = 'utf-8')
    file.write(globals()[f'{name}_e']+' ')
    file.close()
    p = path +'output/'+f'{name}_s_doublespacelinecorpus.txt'
    paths.append(p)
    file = open(p, 'w', encoding = 'utf-8')
    file.write(globals()[f'{name}_s']+' ')
    file.close()
    p = path +'output/'+f'{name}_g_doublespacelinecorpus.txt'
    paths.append(p)
    file = open(p, 'w', encoding = 'utf-8')
    file.write(globals()[f'{name}_g']+' ')
    file.close()

텍스트 파일로 저장된 e/s/g별 문서 불러와서 명사 추출

Soynlp의 LRNounExtractor_v2 이용

In [100]:
nouns = []

In [101]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

for path in paths:
    corpus_path = path
    sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

    noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)
    noun_extractor.train(sents)
    nouns.append(noun_extractor.extract())

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1298 from 527 sents. mem=0.137 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=2103, mem=0.137 Gb
[Noun Extractor] batch prediction was completed for 401 words
[Noun Extractor] checked compounds. discovered 13 compounds
[Noun Extractor] postprocessing detaching_features : 153 -> 153
[Noun Extractor] postprocessing ignore_features : 153 -> 147
[Noun Extractor] postprocessing ignore_NJ : 147 -> 147
[Noun Extractor] 147 nouns (13 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.137 Gb                    
[Noun Extractor] 33.48 % eojeols are covered
[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 2503 from 1041 sents. mem

In [102]:
# 빈도수만큼 해당 명사를 리스트에 저장

for i, n in enumerate(nouns):
    globals()[f"keys_{file_list[i//3]}_{i%3}"] = []
    for w, v in n.items():
        for j in range(int(v[0])):
            globals()[f"keys_{file_list[i//3]}_{i%3}"].append(w)
    globals()[f"keys_{file_list[i//3]}_{i%3}"] = [k for k in locals()[f"keys_{file_list[i//3]}_{i%3}"] if k.isalpha()]
    globals()[f"keys_{file_list[i//3]}_{i%3}"] = [k for k in locals()[f"keys_{file_list[i//3]}_{i%3}"] if not re.match(r"\d{4}년", k)]
    globals()[f"keys_{file_list[i//3]}_{i%3}"] = [k for k in locals()[f"keys_{file_list[i//3]}_{i%3}"] if not re.search(r'현대홈쇼핑|GS|롯데|신세계|LG', k)]

In [104]:
for n in range(len(file_list)):
    globals()[f'{file_list[n]}_e'] = [i for i in globals()[f'keys_{file_list[n]}_0'] if len(i)>2 and not i.isdigit()]
    globals()[f'{file_list[n]}_s'] = [i for i in globals()[f'keys_{file_list[n]}_1'] if len(i)>2 and not i.isdigit()]
    globals()[f'{file_list[n]}_g'] = [i for i in globals()[f'keys_{file_list[n]}_2'] if len(i)>2 and not i.isdigit()]

TF-IDF 계산

In [106]:
docs = []
for n in range(len(file_list)):
    docs.append(globals()[f'{file_list[n]}_e'])
    docs.append(globals()[f'{file_list[n]}_s'])
    docs.append(globals()[f'{file_list[n]}_g'])

In [107]:
def tf(t, d):
  return d.count(t)

def idf(t):
  c = 0
  for doc in docs:
    c += t in doc
  return np.log(len(docs)/(c+sys.float_info.min))

def tfidf(t, d):
  return tf(t,d)* idf(t)

In [109]:
corps = []
for n in range(len(file_list)):
    corp = pd.DataFrame(globals()[f'{file_list[n]}_e'],columns= ['word'])
    corps.append((f'{file_list[n]}',corp,globals()[f'{file_list[n]}_e']))
    corp = pd.DataFrame(globals()[f'{file_list[n]}_s'],columns= ['word'])
    corps.append((f'{file_list[n]}',corp,globals()[f'{file_list[n]}_s']))
    corp = pd.DataFrame(globals()[f'{file_list[n]}_g'],columns= ['word'])
    corps.append((f'{file_list[n]}',corp,globals()[f'{file_list[n]}_g']))

In [111]:
# 계산 및 내림차순으로 정렬

for c in corps:
    for i, item in c[1].iterrows():
        c[1].loc[i,'tf-idf'] = tfidf(item['word'],c[2])
    c[1].drop_duplicates(inplace = True)
    c[1].sort_values(by = 'tf-idf', ascending=False, inplace=True)
    c[1].reset_index(inplace=True, drop = True)

In [112]:
for c in corps:
    print(c[0], len(c[1]))

ESG_text_GSretail_2023.csv 38
ESG_text_GSretail_2023.csv 133
ESG_text_GSretail_2023.csv 52
ESG_text_hyundaehomeshopping_2023.csv 49
ESG_text_hyundaehomeshopping_2023.csv 148
ESG_text_hyundaehomeshopping_2023.csv 26
ESG_text_LGlife_2023.csv 75
ESG_text_LGlife_2023.csv 197
ESG_text_LGlife_2023.csv 20
ESG_text_lottehi_2023.csv 24
ESG_text_lottehi_2023.csv 111
ESG_text_lottehi_2023.csv 23
ESG_text_sinsegye_2023.csv 50
ESG_text_sinsegye_2023.csv 107
ESG_text_sinsegye_2023.csv 24


TF-IDF 점수가 1보다 큰 단어만 코퍼스로 구축

In [113]:
esg = ('e','s','g','e','s','g','e','s','g')
# corp = ((e_corpus,10), (s_corpus,10), (g_corpus,10))
for c in corps:
    name = c[0].replace('ESG_text_','').replace('.csv','')
    globals()[f'{name}corpus'] = pd.DataFrame(columns=['word', 'label', 'tf-idf'])

for i, c in zip(esg, corps):
    name = c[0].replace('ESG_text_','').replace('.csv','')
    # corpus = pd.concat([corpus,c[0][c[0]['tf-idf']>c[1]]])
    globals()[f'{name}corpus'] = pd.concat([globals()[f'{c[0]}corpus'],c[1]])
    globals()[f'{name}corpus'] = globals()[f'{c[0]}corpus'].reset_index(drop=True)
    globals()[f'{name}corpus'].loc[len(globals()[f'{c[0]}corpus'])-1,'label'] = i
    print(f'{name}corpus', len(globals()[f'{c[0]}corpus']))

for file in file_list:
    name = file.replace('ESG_text_','').replace('.csv','')
    # 기업명 관련 키워드 제거
    # globals()[f'{name}corpus'] = globals()[f'{name}corpus'][~globals()[f'{name}corpus']['word'].str.contains(r"현대홈쇼핑|GS|롯데|신세계|LG", regex=True)]
    # print(len(globals()[f'{name}corpus']))
    globals()[f'{name}corpus'] = globals()[f'{name}corpus'].fillna(method = 'bfill')
    globals()[f'{name}corpus'] = globals()[f'{name}corpus'].dropna()
    globals()[f'{name}corpus'] = globals()[f'{name}corpus'].drop_duplicates()
    globals()[f'{name}corpus'] = globals()[f'{name}corpus'].reset_index(drop = True)
    path = './1-2. ESG report/'
    if not os.path.isdir(path + 'output/'):
        os.mkdir(path + 'output/')
    globals()[f'{name}corpus'].to_csv(path + 'output/'+ 'corpus_esg_keyword_v4_' + name +'.csv', index=False, encoding='utf-8-sig')

GSretail_2023corpus 1805
GSretail_2023corpus 1805
GSretail_2023corpus 1805
hyundaehomeshopping_2023corpus 1798
hyundaehomeshopping_2023corpus 1798
hyundaehomeshopping_2023corpus 1798
LGlife_2023corpus 2357
LGlife_2023corpus 2357
LGlife_2023corpus 2357
